## 💡출력 파서 (Output Parser)
* LLM의 출력을 구조화하는 컴포넌트
* 출력을 원하는 형식으로 변환하여 보기 쉽게 만들거나, 구조화된 데이터 생성이 필요한 경우 사용

### 📌01. Pydantic Output Parser

Pydantic Output Parser 사용을 위해서는 두 가지 메서드가 필요하다.(대부분의 OutputParser가 필요로 함)

1. ```get_format_instructions()```
* 언어 모델이 출력해야 할 결과물의 형식을 정의해주는 지침을 반환하는 메서드
* 만약, JSON 형식을 준수하는 결과를 출력해야 하는 경우라면, JSON 형식에 맞게 결과물을 구조화할 수 있도록 지시하는 내용이 반환된다.

2. ```parse()```
* 언어 모델의 출력을 받아들여 이를 parser의 타입에 맞게 변환하는 역할을 수행하는 메서드
* 사전에 정의된 스키마를 반영하여 parser를 생성한 후, parse() 메서드를 통해 형식을 변환하여 원하는 데이터 형식으로 구조화할 수 있다.


In [131]:
from langchain_teddynote.messages import stream_response

In [163]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from pprint import *

llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [149]:
email_conversation = """ 
안녕하세요 과장님 
오늘 너무 바빠서 전화를 드리기 어려웠습니다.
일단 주신 파일을 보는데 차이점이 주요하게는 BASIC과 EXCLUSIVE가 주요 포인트인 것 같아요 
HCX와 HCX-DASH는 민간 공공 모두에 약간의 입력토큰 차이만 있으니까요

그럼 추가적으로 궁금한 것이 BASIC과 EXCLUSIVE에서의 보안 부분에서는 어떤 차이가 있을까요?
BASIC으로 활용하면 보안성이 떨어지는 기술이 적용되거나 그런 것이 아닌지요?

그리고 BASIC을 활용하는 것에 대한 사례, EXCLUSIVE를 활용하는 사례는 어떤 것이 있는지 대략적으로라도 알고 싶습니다.
(제 개인적인 생각에는 요약서비스는 개인정보가 저장 여부도 중요한 것 같은데...
다채움에서는  과제 요약서비스로 네이버 클라우드를 활용 시에는 개인정보를 활용하지도 않고, 요약한 정보가 휘발될 것 같은데 이러한 사례에서도 공공존을 활용하는 것이 맞는지 기술적인 검토나 제안 등을 해주시면 감사하겠습니다. 
경북교육청은 어떤 서비스를 활용하고 있는지 저희가 참고할 수 있으면 더욱 좋을 것 같습니다. )

"""

In [151]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

# 우리가 평소에 일반적인 방법으로 LLM을 사용할 때 받게되는 응답의 형식으로 나타남
output = stream_response(answer, return_output=True)

안녕하세요 과장님,

이메일의 중요한 내용은 다음과 같습니다:

1. BASIC과 EXCLUSIVE의 주요 차이점에 대한 문의:
   - 보안 부분에서의 차이점
   - BASIC을 활용할 경우 보안성이 떨어지는 기술이 적용되는지 여부

2. BASIC과 EXCLUSIVE 활용 사례에 대한 문의:
   - BASIC과 EXCLUSIVE 각각의 활용 사례
   - 개인정보 저장 여부와 관련된 요약 서비스의 사례

3. 기술적 검토 및 제안 요청:
   - 네이버 클라우드를 활용한 과제 요약 서비스에서의 개인정보 활용 여부
   - 공공존 활용의 적절성에 대한 기술적 검토 및 제안

4. 경북교육청의 서비스 활용 사례에 대한 참고 요청

위의 결과물은, 우리가 일반적으로 LLM에(ChatGPT 등) 메일 요약을 요청한 경우 받을 수 있는 응답 형태이다.<br>
하지만, 요청할 때마다 응답의 형태(형식)가 달라질 가능성이 높기 때문에, 애플리케이션 데이터로 활용하기엔 어려움이 있다.

Pydantic을 사용하여 파싱하게 되면, LLM의 응답을 미리 정의해둔 구조대로 일관되게(Json 형식으로) 받을 수 있다.

In [184]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람", default="보낸이 미상")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소", default="알 수 없음")
    subject: str = Field(description="메일 제목", default="알 수 없음")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 due date", default="기한 없음")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [185]:
# instruction 을 출력
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"default": "보낸이 미상", "description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"default": "알 수 없음", "description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"default": "알 수 없음", "description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"default": "기한 없음", "description": "메일 본문에 언급된 due date", "title": "Date", "type": "string"}}, "required": ["summary"]}
```


### 📌부분변수(partial_variables)
고정된 값(변경되지 않는 값)을 미리 설정하는 기능<br>
사용자가 입력할 때마다 변경되는 input_variables와 구분하여 템플릿을 설정한다.<br>

만약, partial_variables를 사용하지 않는다면, format_instructions 역시 input_variables에 포함되어야 하며,<br>
이 경우 프롬프트를 실행할 때마다 format_instructions 값을 매번 직접 넣어주어야 한다.

In [190]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

In [191]:
# 전달되는 instruction 내용 확인해보기
# 형식 지시사항에 앞서 정의한 Class의 내용이 반영되었음을 확인할 수 있다.
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"default": "보낸이 미상", "description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"default": "알 수 없음", "description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"default": "알 수 없음", "description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"default": "기한 없음", "description": "메일 본문에 언급된 due date", "title": "Date", "type": "string"}}, "required": ["summary"]}
```


In [192]:
# chain 을 생성합니다.
chain = prompt | llm

In [193]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
output = stream_response(response, return_output=True)

```json
{
    "person": "보낸이 미상",
    "email": "알 수 없음",
    "subject": "알 수 없음",
    "summary": "보낸이는 BASIC과 EXCLUSIVE의 주요 차이점에 대해 문의하고 있으며, 특히 보안성 차이에 대해 궁금해하고 있습니다. 또한 BASIC과 EXCLUSIVE의 활용 사례에 대한 정보를 요청하고 있습니다. 개인정보 저장 여부와 관련된 요약 서비스의 기술적 검토 및 제안도 요청하고 있으며, 경북교육청의 서비스 활용 사례에 대한 참고를 원하고 있습니다.",
    "date": "기한 없음"
}
```

In [194]:
# PydanticOutputParser 를 사용하여 결과를 파싱합니다.
structured_output = parser.parse(output)
print(structured_output)

person='보낸이 미상' email='알 수 없음' subject='알 수 없음' summary='보낸이는 BASIC과 EXCLUSIVE의 주요 차이점에 대해 문의하고 있으며, 특히 보안성 차이에 대해 궁금해하고 있습니다. 또한 BASIC과 EXCLUSIVE의 활용 사례에 대한 정보를 요청하고 있습니다. 개인정보 저장 여부와 관련된 요약 서비스의 기술적 검토 및 제안도 요청하고 있으며, 경북교육청의 서비스 활용 사례에 대한 참고를 원하고 있습니다.' date='기한 없음'


In [195]:
# 출력 파서를 추가하여 전체 체인을 재구성합니다.
chain = prompt | llm | parser

In [196]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력됩니다.
response

EmailSummary(person='보낸이 미상', email='알 수 없음', subject='알 수 없음', summary='이메일 발신자는 BASIC과 EXCLUSIVE의 주요 차이점에 대해 문의하고 있으며, 특히 보안 측면에서의 차이를 알고 싶어합니다. 또한 BASIC과 EXCLUSIVE의 활용 사례에 대한 정보를 요청하고 있습니다. 개인정보 저장 여부와 관련된 요약 서비스의 기술적 검토 및 제안도 필요하다고 언급하고 있으며, 경북교육청의 활용 사례에 대한 참고를 원하고 있습니다.', date='기한 없음')

In [197]:
llm_with_structered = ChatOpenAI(
    temperature=0, model_name="gpt-4o"
).with_structured_output(EmailSummary)

In [199]:
# invoke() 함수를 호출하여 결과를 출력합니다.
# https://docs.pydantic.dev/latest/concepts/fields/#default-values
answer = llm_with_structered.invoke(email_conversation)
answer

c:\Users\Goodusdata\Desktop\jy\AI Study\langchain\Lib\site-packages\langchain_openai\chat_models\base.py:410: UserWarning: Invalid schema for OpenAI's structured output feature, which is the default method for `with_structured_output` as of langchain-openai==0.3. Specify `method="function_calling"` instead or update your schema. See supported schemas: https://platform.openai.com/docs/guides/structured-outputs#supported-schemas
  warnings.warn(message)


BadRequestError: Error code: 400 - {'error': {'message': "Invalid schema for response_format 'EmailSummary': In context=('properties', 'person'), 'default' is not permitted.", 'type': 'invalid_request_error', 'param': 'response_format', 'code': None}}

## 📌02. CommaSeparatedListOutputParser

쉼표로 구분된 항목 목록을 반환할 필요가 있을 때 유용한 파서<br>
요청한 정보를 쉼표로 구분하여 간결한 형태로 제공받을 수 있다. <br><br>

### 🐼예시
* 모델이 출력하는 답변 : "apple, grape, banana"
* CommaSeparatedListOutputParser 사용 시 : ["apple", "grape", "banana"]

In [5]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()


# 출력 형식 지침 가져오기 -> get_format_instructions는 모델에게 전달할 Instructions를 반환한다.
format_instructions = output_parser.get_format_instructions()

format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

### 📌만약, subject 가 fruits 라면?

아래와 같은 프롬프트가 구성된다.

```
List five fruits.
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz` 
```

In [12]:
# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    # 주제에 대한 다섯 가지를 나열하라는 템플릿
    # {subject} : 사용자가 입력할 주제가 들어갈 자리
    # {format_instructions} : 앞서 반환된 출력 형식 지침을 프롬프트 템플릿에 함께 제공
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],  # 입력 변수로 'subject' 사용
    # 부분 변수로 형식 지침 사용
    partial_variables={"format_instructions": format_instructions},
)

In [13]:
# ChatOpenAI 모델 초기화
model = ChatOpenAI(temperature=0)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

In [14]:
# "스위스 관광명소"에 대한 체인 호출 실행
chain.invoke({"subject": "스위스 관광명소"})

['융프라우', '취리히', '루체른', '제네바', '베른']

In [15]:
# 스트림을 순회합니다.
for s in chain.stream({"subject": "스위스 관광명소"}):
    print(s)  # 스트림의 내용을 출력합니다.

['융프라우']
['취리히']
['루체른']
['인터라켄']
['제네바']


## 📌03. StructuredOutputParser
LLM의 응답을 dict 형식으로 구성하여 key-value 쌍으로 반환받을 수 있다.<br>
`Pydantic/JSON` 파서의 경우 GPT 등 강력한 모델을 활용하는 경우 더 유용하며, `StructuredOutputParser`는 <U>파라미터 수가 더 적은 모델에 유용</U>하다.

In [16]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

### 🐼ResponseSchema 클래스
`StructuredOutputParser`의 dict 구조에서, key-value 쌍에 각각 어떤 값들이 들어가야 하는지 정의하는 데 사용된다.<br>

속성
* `name` : 응답 필드의 이름(dict key)
* `description` : 응답 필드에 대한 설명(dict value)

In [17]:
# 사용자의 질문에 대한 답변

'''
응답의 형태
{'answer' : '[사용자의 질문에 대한 답변]', 'source' : '[사용자의 질문에 답할 때 사용한 웹사이트 주소]'}
'''

response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]
# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [19]:
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // 사용자의 질문에 대한 답변\n\t"source": string  // 사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.\n}\n```'

### 프롬프트 형태
```
answer the users question as best as possible.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // 사용자의 질문에 대한 답변\n\t"source": string  // 사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.\n}\n```
{사용자의 질문}
```

In [ ]:

prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

In [20]:
model = ChatOpenAI(temperature=0)  # ChatOpenAI 모델 초기화
chain = prompt | model | output_parser  # 프롬프트, 모델, 출력 파서를 연결

In [21]:
# 캐나다의 수도가 무엇인지 질문합니다.
chain.invoke({"question": "캐나다의 수도는 어디인가요?"})

{'answer': '캐나다의 수도는 오타와입니다.',
 'source': 'https://ko.wikipedia.org/wiki/%EC%98%A4%ED%83%80%EC%99%80'}

In [23]:
for s in chain.stream({"question": "이순신의 업적은 무엇인가요?"}):
    # 스트리밍 출력
    print(s)

{'answer': '이순신은 조선시대의 무신 이순신 장군으로서, 임진왜란에서 일본군을 물리치고 전략적 승리를 거두었습니다.', 'source': 'https://ko.wikipedia.org/wiki/%EC%9D%B4%EC%88%9C%EC%8B%A0'}


## 📌04. JsonOutputParser
사용자가 원하는 JSON 스키마를 지정할 수 있으며, 해당 스키마에 맞게 결과를 출력할 수 있다.

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [25]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

### 🐼BaseModel
`BaseModel`을 상속받은 클래스는 자동으로 데이터 타입 검증 기능을 갖는다.<br>
아래 Topic은 description과 hashtags가 문자열(str) 타입이라는 규칙을 정의한 것이며, 자동으로 타입이 검증된다.

In [28]:
# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [29]:
# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser  # 체인을 구성합니다.

chain.invoke({"question": question})  # 체인을 호출하여 쿼리 실행

{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 인간 활동에 의해 발생하는 온실가스 배출이 주요 원인입니다. 이는 극지방의 빙하 감소, 해수면 상승, 기후 패턴 변화 등 심각한 환경적 영향을 초래합니다.',
 'hashtags': '#지구온난화 #기후변화 #환경보호'}

In [ ]:
class Email(BaseModel):
    #send : str = Field("보낸이 미상", description="메일을 보낸 사람의 이름")
    send : str = Field(description="메일을 보낸 사람의 이름")
    content: str = Field(description="메일의 핵심 내용을 한줄로 정리")
    receive: str = Field(description="메일 받은 사람 이름")
    follow_up: str = Field(description="필요한 후속 조치")
    

In [ ]:
# 질의 작성
question = """ 

안녕하세요. 김민성 매니저님.
굿어스데이터 진재영입니다.

네이버클라우드플랫폼은 장비 세팅 완료 이전에 터널링을 생성하는 기능을 지원하지 않고 있습니다.
따라서, 13시에 본사 장비 세팅 완료하신 후 연락 주시면 터널링 생성 진행하도록 하겠습니다.

또한, 민간존 IPSec VPN Pre-Shared Key에는 특수문자 사용이 불가합니다. 
따라서 특수문자가 포함되지 않은 Pre-Shared Key 전달 부탁드립니다.

[살루스케어 NCP IP 정보]
VPC : 10.0.0.0/16
IPSec VPN Gateway : 175.106.107.89

추가로, IPSec VPN 연동하시고자 하는 특정 서버(서브넷)가 있으신지 확인 요청드립니다.
우선 전체 VPC 대역으로 전달드리오나, 특정 서버에 대해서만 연동을 원하신다면, 해당 서브넷 IP 대역으로 다시 전달드리도록 하겠습니다. 

확인 부탁드립니다.

감사합니다.
진재영 드림

"""

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Email)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 주어진 메일을 형식에 맞게 요약하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser  # 체인을 구성합니다.

chain.invoke({"question": question})  # 체인을 호출하여 쿼리 실행

{'send': '보낸이 미상',
 'content': '네이버클라우드플랫폼은 장비 세팅 완료 후 터널링 생성 가능하며, Pre-Shared Key에 특수문자 사용 불가.',
 'receive': '김민성 매니저',
 'follow_up': '장비 세팅 완료 후 연락 및 특수문자 없는 Pre-Shared Key 전달, 특정 서버 서브넷 IP 대역 확인 요청.'}

## 📌05. PandasDataFrameOutputParser
Pandas Dataframe은 Python으로 2차원 데이터를 다룰 때 사용되는데, <br>
PandasDataFrameOutputParser를 사용하면 데이터프레임에서 데이터를 뽑아 정리된 형태로 볼 수 있다. 

In [44]:
import pprint
from typing import Any, Dict

import pandas as pd
import seaborn as sns # 예시 데이터셋 가져오기 위한 용도
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [45]:
# ChatOpenAI 모델 초기화 (gpt-3.5-turbo 모델 사용을 권장합니다)
model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [50]:
# 출력 목적으로만 사용됩니다.
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    # 파서 출력의 키들을 순회합니다.
    for key in parser_output.keys():
        # 각 키의 값을 딕셔너리로 변환합니다.
        parser_output[key] = parser_output[key].to_dict()
    # 예쁘게 출력합니다.
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

In [ ]:
# 'penguins' 데이터셋 로드
df = sns.load_dataset('penguins')
df.head()

# 펭귄의 종 / 서식하는 섬 / 부리 길이 / 부리 깊이 / 날개 길이 / 체중 / 성별

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [48]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = PandasDataFrameOutputParser(dataframe=df)

# 파서의 지시사항을 출력합니다.
print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [55]:
# 열 작업 예시입니다.
df_query = "island column 을 조회해 주세요."


# 프롬프트 템플릿을 설정합니다.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # 입력 변수 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },  # 부분 변수 설정
)

# 체인 생성
chain = prompt | model | parser

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 출력
format_parser_output(parser_output)

{'island': {0: 'Torgersen',
            1: 'Torgersen',
            2: 'Torgersen',
            3: 'Torgersen',
            4: 'Torgersen',
            5: 'Torgersen',
            6: 'Torgersen',
            7: 'Torgersen',
            8: 'Torgersen',
            9: 'Torgersen',
            10: 'Torgersen',
            11: 'Torgersen',
            12: 'Torgersen',
            13: 'Torgersen',
            14: 'Torgersen',
            15: 'Torgersen',
            16: 'Torgersen',
            17: 'Torgersen',
            18: 'Torgersen',
            19: 'Torgersen',
            20: 'Biscoe',
            21: 'Biscoe',
            22: 'Biscoe',
            23: 'Biscoe',
            24: 'Biscoe',
            25: 'Biscoe',
            26: 'Biscoe',
            27: 'Biscoe',
            28: 'Biscoe',
            29: 'Biscoe',
            30: 'Dream',
            31: 'Dream',
            32: 'Dream',
            33: 'Dream',
            34: 'Dream',
            35: 'Dream',
            36: 'Dre

In [64]:
# 행 조회 예시입니다.
# df_query = "Retrieve the first row."
df_query = "0번째 행 출력"

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
format_parser_output(parser_output)

{'0': {'bill_depth_mm': 18.7,
       'bill_length_mm': 39.1,
       'body_mass_g': 3750.0,
       'flipper_length_mm': 181.0,
       'island': 'Torgersen',
       'sex': 'Male',
       'species': 'Adelie'}}


In [74]:
df['body_mass_g'].head().mean()
#print(df['body_mass_g'].iloc[:5].mean())

3562.5

In [ ]:
# 임의의 Pandas DataFrame 작업 예시, 행의 수를 제한합니다.
df_query = "Retrieve the average of the body_mass_g from row 0 to 4." 

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
print(parser_output)

{'mean': 3562.5}


In [77]:
#df_query = "Calculate average body_mass_g rate" # **중요한점 : 올바르게 형식화된, 명확한 쿼리를 사용하는 것이 중요!!
df_query = "Calculate the average of the 'body_mass_g' column."

# 체인 실행
parser_output = chain.invoke({"query": df_query})

# 결과 출력
print(parser_output)

{'mean': 4201.754385964912}


In [78]:
df['body_mass_g'].mean()

4201.754385964912

## 📌06. DatetimeOutputParser
LLM의 출력을 `datetime` 형식으로 파싱하는데 활용 가능

### 🐼날짜 및 시간 형식 코드

| 형식 코드 | 설명               | 예시                     |
|----------|------------------|-------------------------|
| `%Y`     | 4자리 연도        | `2024`                  |
| `%y`     | 2자리 연도        | `24`                    |
| `%m`     | 2자리 월          | `07`                    |
| `%d`     | 2자리 일          | `04`                    |
| `%H`     | 24시간제 시간     | `14`                    |
| `%I`     | 12시간제 시간     | `02`                    |
| `%p`     | AM 또는 PM       | `PM`                    |
| `%M`     | 2자리 분          | `45`                    |
| `%S`     | 2자리 초          | `08`                    |
| `%f`     | 마이크로초 (6자리) | `000123`                |
| `%z`     | UTC 오프셋        | `+0900`                 |
| `%Z`     | 시간대 이름       | `KST`                    |
| `%a`     | 요일 약어         | `Thu`                   |
| `%A`     | 요일 전체         | `Thursday`              |
| `%b`     | 월 약어           | `Jul`                   |
| `%B`     | 월 전체           | `July`                  |
| `%c`     | 전체 날짜와 시간  | `Thu Jul 4 14:45:08 2024` |
| `%x`     | 전체 날짜         | `07/04/24`              |
| `%X`     | 전체 시간         | `14:45:08`              |



In [18]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [19]:
# 날짜 및 시간 출력 파서
output_parser = DatetimeOutputParser()
output_parser.format = "%Y-%m-%d"
#output_parser.format = "%c"

In [20]:
# 사용자 질문에 대한 답변 템플릿
template = """Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    },  # 지침을 템플릿에 적용
)

# 프롬프트 내용을 출력
prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%d'.\n\nExamples: 1562-10-11, 0602-12-21, 1650-11-06\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:')

In [21]:
# Chain 을 생성합니다.
chain = prompt | ChatOpenAI() | output_parser

# 체인을 호출하여 질문에 대한 답변을 받습니다.
output = chain.invoke({"question": "Google 이 창업한 연도는?"})

In [ ]:
# datetime 객체 형태로 출력됨
output

datetime.datetime(1998, 9, 4, 0, 0)

In [25]:
# 결과를 문자열로 변환
output.strftime("%Y-%m-%d")

'1998-09-04'

## 📌07. EnumOutputParser
LLM의 응답 값을 제한된 범위 안에서만 허용하도록 설정하는 데에 도움을 줄 수 있는 파서이다.<br>
예상치 못한 응답값을 방지하는 데에 도움이 될 수 있다.<br>
(ex. Y/N로 대답해야 하는 경우, Task의 진행상태(성공, 실패, 진행중)를 출력해야 하는 경우 등)

### 🐼Enum?
Enumeration(열거형)이란, 서로 관련있는 여러 개의 상수 집합을 정의할 때 사용하는 모듈이다.<br>


In [26]:
from langchain.output_parsers.enum import EnumOutputParser
from enum import Enum

In [28]:
class Colors(Enum):
    RED = "빨간색"
    GREEN = "초록색"
    BLUE = "파란색"

In [31]:
type(Colors.BLUE)

<enum 'Colors'>

In [32]:
# EnumOutputParser 인스턴스 생성
parser = EnumOutputParser(enum=Colors)

In [33]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template(
    """다음의 물체는 어떤 색깔인가요?

Object: {object}

Instructions: {instructions}"""
    # 파서에서 지시사항 형식을 가져와 부분적으로 적용합니다.
).partial(instructions=parser.get_format_instructions())
# 프롬프트와 ChatOpenAI, 파서를 연결합니다.
chain = prompt | ChatOpenAI() | parser

In [36]:
response = chain.invoke({"object": "하늘"})  # "하늘" 에 대한 체인 호출 실행
print(response)

Colors.BLUE


또다른 예시

In [38]:
class Reviews(Enum):
    POSITIVE = "대체로 긍정적"
    NEUTRAL = "호불호가 갈림"
    NEGATIVE = "대체로 부정적"

In [43]:
# EnumOutputParser 인스턴스 생성
parser = EnumOutputParser(enum=Reviews)

In [46]:
# 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template(
    """다음 영화의 평점은 어떤가요?

Object: {object}

Instructions: {instructions}"""
    # 파서에서 지시사항 형식을 가져와 부분적으로 적용합니다.
).partial(instructions=parser.get_format_instructions())
# 프롬프트와 ChatOpenAI, 파서를 연결합니다.
chain = prompt | ChatOpenAI() | parser

In [ ]:
response = chain.invoke({"object": "The Shawshank Redemption"})  # 쇼생크 탈출
print(response)

Reviews.POSITIVE


In [ ]:
response = chain.invoke({"object": "Eternals"})  # 이터널스
print(response)

Reviews.NEUTRAL


In [ ]:
response = chain.invoke({"object": "Mars Needs Moms"})  # 화성은 엄마가 필요해(2011)
print(response)

Reviews.NEGATIVE


## 📌08. OutputFixingParser
출력 파싱 과정에서 발생할 수 있는 오류를 자동으로 수정하는 기능을 제공한다.<br>
파서가 처리할 수 없는 형식이나 오류를 반환하는 경우, LLM을 추가적으로 호출하여 오류를 수정할 수 있다.

예를 들어, 첫 번째 결과에서 정의된 스키마를 준수하지 않는 결과가 도출될 경우,<br>
`OutputFixingParser`가 자동으로 출력 형식이 잘못되었음을 인식하고 이를 수정하기 위한 내용을 추가하여 LLM에 다시 응답을 요청한다.



In [112]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from langchain.output_parsers import OutputFixingParser

In [128]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List


class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(
        description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

In [129]:
# 잘못된 형식을 일부러 입력
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

# 잘못된 형식으로 입력된 데이터를 파싱하려고 시도
parser.parse(misformatted)

# 오류 출력

OutputParserException: Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [125]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [126]:
# 잘못된 형식의 출력
misformatted

"{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

In [127]:
# OutputFixingParser 를 사용하여 잘못된 형식의 출력을 파싱
actor = new_parser.parse(misformatted)

KeyError: "Input to PromptTemplate is missing variables {'instructions'}.  Expected: ['completion', 'error', 'instructions'] Received: ['completion', 'error']\nNote: if you intended {instructions} to be part of the string and not a variable, please escape it with double curly braces like: '{{instructions}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [118]:
# 파싱된 결과
actor

NameError: name 'actor' is not defined